In [10]:
import pandas as pd
data = pd.read_excel('Example.xlsx',sheet_name='data')

In [11]:
data.groupby(['MATERIAL']).agg(
    count_prodorder = ('PRODORDER','nunique')
).reset_index()

,MATERIAL,count_prodorder
0,jeanpantl,2
1,jeanpantm,1
2,slimfitpantl,2
3,slimfitpantm,2
4,slimfitpants,1
5,tshirtwhl,2
6,tshirtwhm,2


In [8]:
data.groupby(['PRODORDER']).agg(
    count_material = ('MATERIAL','nunique')
).reset_index()

,PRODORDER,count_material
0,1002,1
1,1003,1
2,1012,1
3,1013,1
4,1014,1
5,1021,1
6,1022,1
7,1023,1
8,2002,1
9,2003,1


---

In [1]:
import pandas as pd
data = pd.read_excel('Example.xlsx',sheet_name='data')
data

,ID,MATERIAL,PLANT,DESCRIPTION,PRODORDER,QUANTITY,UNIT,AMOUNT,UDATE
0,1,tshirtwhm,1,T-shirt White Size M,1002,5000,PC,50000,2025-01-01 08:00:00
1,2,jeanpantm,2,Jean Pant Size M,1012,100,PC,20000,2025-01-01 08:02:13
2,3,tshirtwhm,1,T-shirt White Size M,2002,10000,PC,100000,2025-01-01 10:13:37
3,4,tshirtwhm,1,T-shirt White Size M,2002,-10000,PC,-100000,2025-01-01 10:14:01
4,5,slimfitpantm,1,Slim Fit Pant Size M,1022,300,PC,60000,2025-01-01 11:03:22
5,6,slimfitpantm,1,Slim Fit Pant Size M,2022,400,PC,80000,2025-01-01 11:03:22
6,7,slimfitpants,1,Slim Fit Pant Size S,1021,200,PC,40000,2025-01-02 11:03:40
7,8,slimfitpantl,1,Slim Fit Pant Size L,1023,600,PC,120000,2025-01-02 11:20:25
8,9,slimfitpantl,1,Slim Fit Pant Size L,1023,600,PC,120000,2025-01-02 11:40:00
9,10,slimfitpantl,1,Slim Fit Pant Size L,2023,600,PC,120000,2025-01-02 11:40:30


In [2]:
cancel_list = data.loc[data['QUANTITY'] < 0, 'PRODORDER'].unique()
data_cancel = data[data['PRODORDER'].isin(cancel_list)].copy()

In [ ]:
data_cancel['ABS_QUANTITY'] = data_cancel['QUANTITY'].abs()

mapping_df=pd.DataFrame(columns=['PLUS_PRODUCE_ID','MINUS_PRODUCE_ID'])

In [4]:
for cancel in cancel_list:
    data_i = data_cancel[data_cancel['PRODORDER'] == cancel]

    data_i_minus = data_i[data_i['QUANTITY']<0]
    minus_id_list = data_i_minus['ID'].unique()

    for minus_id in minus_id_list:
        abs_qty = data_i_minus[data_i_minus['ID']==minus_id]['ABS_QUANTITY'].unique()[0]

        data_ij = data_i[(data_i['ID'] == minus_id) | (data_i['QUANTITY']>0)].copy()
        
        mapping_id_i = data_ij[
            (data_ij['ID'] < minus_id) 
            & (data_ij['ABS_QUANTITY'] == abs_qty)
            & (~ data_ij['ID'].isin(mapping_df['PLUS_PRODUCE_ID'].unique()))                  
                            ].sort_values(by=['ID'],ascending=True)['ID'].iloc[0]

        mapping_df_i = pd.DataFrame({
                        'PLUS_PRODUCE_ID':[mapping_id_i],
                        'MINUS_PRODUCE_ID':[minus_id]
                    })

        mapping_df = pd.concat([mapping_df,mapping_df_i],ignore_index=True)

mapping_df

,PLUS_PRODUCE_ID,MINUS_PRODUCE_ID
0,3,4
1,6,17
2,8,18
3,9,22
4,10,19
5,12,20
6,13,21
